In [ ]:
import requests
import json
import time
from datetime import datetime, date

class COVID19API:
    def __init__(self):
        self.base_url = "https://disease.sh/v3/covid-19"
        self.headers = {
            'User-Agent': 'COVID19APIScraper/1.0 (https://example.com/contact)'
        }
    
    def get_global_data(self):
        url = f"{self.base_url}/all"
        
        try:
            response = requests.get(url, headers=self.headers)
            if response.status_code == 200:
                return response.json()
            else:
                print(f"Error getting global data: {response.status_code}")
                return None
        except Exception as e:
            print(f"Exception getting global data: {e}")
            return None
    
    def get_countries_data(self):
        url = f"{self.base_url}/countries"
        params = {
            'yesterday': 'false',
            'twoDaysAgo': 'false',
            'sort': 'cases',
            'allowNull': 'true'
        }
        
        try:
            response = requests.get(url, params=params, headers=self.headers)
            if response.status_code == 200:
                return response.json()
            else:
                print(f"Error getting countries data: {response.status_code}")
                return []
        except Exception as e:
            print(f"Exception getting countries data: {e}")
            return []
    
    def get_country_data(self, country):
        url = f"{self.base_url}/countries/{country}"
        params = {
            'yesterday': 'false',
            'twoDaysAgo': 'false',
            'strict': 'true',
            'allowNull': 'true'
        }
        
        try:
            response = requests.get(url, params=params, headers=self.headers)
            if response.status_code == 200:
                return response.json()
            else:
                print(f"Error getting data for {country}: {response.status_code}")
                return None
        except Exception as e:
            print(f"Exception getting data for {country}: {e}")
            return None
    
    def get_country_historical_data(self, country, days='all'):
        url = f"{self.base_url}/historical/{country}"
        params = {
            'lastdays': days
        }
        
        try:
            response = requests.get(url, params=params, headers=self.headers)
            if response.status_code == 200:
                return response.json()
            else:
                print(f"Error getting historical data for {country}: {response.status_code}")
                return None
        except Exception as e:
            print(f"Exception getting historical data for {country}: {e}")
            return None
    
    def get_vaccine_data(self, country=None):
        if country:
            url = f"{self.base_url}/vaccine/coverage/countries/{country}"
        else:
            url = f"{self.base_url}/vaccine/coverage"
        
        params = {
            'lastdays': 'all',
            'fullData': 'false'
        }
        
        try:
            response = requests.get(url, params=params, headers=self.headers)
            if response.status_code == 200:
                return response.json()
            else:
                print(f"Error getting vaccine data: {response.status_code}")
                return None
        except Exception as e:
            print(f"Exception getting vaccine data: {e}")
            return None
    
    def process_historical_data(self, historical_data, country_name):
        if not historical_data or 'timeline' not in historical_data:
            return None
        
        timeline = historical_data['timeline']
        cases = timeline.get('cases', {})
        deaths = timeline.get('deaths', {})
        recovered = timeline.get('recovered', {})

        daily_data = []
        dates = sorted(cases.keys())
        
        for i, date_str in enumerate(dates):
            current_cases = cases.get(date_str, 0)
            current_deaths = deaths.get(date_str, 0)
            current_recovered = recovered.get(date_str, 0)

            if i > 0:
                prev_date = dates[i-1]
                new_cases = current_cases - cases.get(prev_date, 0)
                new_deaths = current_deaths - deaths.get(prev_date, 0)
                new_recovered = current_recovered - recovered.get(prev_date, 0)
            else:
                new_cases = current_cases
                new_deaths = current_deaths
                new_recovered = current_recovered
            
            daily_data.append({
                'date': date_str,
                'total_cases': current_cases,
                'total_deaths': current_deaths,
                'total_recovered': current_recovered,
                'new_cases': max(0, new_cases),
                'new_deaths': max(0, new_deaths),
                'new_recovered': max(0, new_recovered),
                'active': max(0, current_cases - current_deaths - current_recovered)
            })
        
        return {
            'country': country_name,
            'total_records': len(daily_data),
            'date_range': {
                'start': dates[0] if dates else None,
                'end': dates[-1] if dates else None
            },
            'daily_data': daily_data,
            'latest_data': daily_data[-1] if daily_data else {}
        }
    
    def scrape_countries_data(self, countries):
        print(f"Scraping COVID-19 data for countries: {', '.join(countries)}")
        
        all_data = {}
        
        for i, country in enumerate(countries, 1):
            print(f"\nProcessing country {i}/{len(countries)}: {country}")

            current_data = self.get_country_data(country)

            historical_data = self.get_country_historical_data(country)
            
            if current_data and historical_data:
                country_name = current_data.get('country', country)

                processed_historical = self.process_historical_data(historical_data, country_name)

                combined_data = {
                    'country_info': {
                        'country': country_name,
                        'country_code': current_data.get('countryInfo', {}).get('iso2', ''),
                        'continent': current_data.get('continent', ''),
                        'population': current_data.get('population', 0),
                        'flag': current_data.get('countryInfo', {}).get('flag', ''),
                        'coordinates': {
                            'lat': current_data.get('countryInfo', {}).get('lat', 0),
                            'long': current_data.get('countryInfo', {}).get('long', 0)
                        }
                    },
                    'current_stats': {
                        'updated': datetime.fromtimestamp((current_data.get('updated') or 0) / 1000).isoformat(),
                        'cases': current_data.get('cases') or 0,
                        'today_cases': current_data.get('todayCases') or 0,
                        'deaths': current_data.get('deaths') or 0,
                        'today_deaths': current_data.get('todayDeaths') or 0,
                        'recovered': current_data.get('recovered') or 0,
                        'today_recovered': current_data.get('todayRecovered') or 0,
                        'active': current_data.get('active') or 0,
                        'critical': current_data.get('critical') or 0,
                        'cases_per_million': current_data.get('casesPerOneMillion') or 0,
                        'deaths_per_million': current_data.get('deathsPerOneMillion') or 0,
                        'tests': current_data.get('tests') or 0,
                        'tests_per_million': current_data.get('testsPerOneMillion') or 0
                    },
                    'historical_data': processed_historical,
                    'statistics': {
                        'case_fatality_rate': round(((current_data.get('deaths') or 0) / max(current_data.get('cases') or 1, 1)) * 100, 2),
                        'recovery_rate': round(((current_data.get('recovered') or 0) / max(current_data.get('cases') or 1, 1)) * 100, 2),
                        'active_rate': round(((current_data.get('active') or 0) / max(current_data.get('cases') or 1, 1)) * 100, 2)
                    },
                    'scraped_at': datetime.now().isoformat()
                }
                
                all_data[country] = combined_data
                
                print(f"  Total cases: {current_data.get('cases', 0):,}")
                print(f"  Deaths: {current_data.get('deaths', 0):,}")
                print(f"  Historical records: {processed_historical['total_records'] if processed_historical else 0}")
                print(f"  Date range: {processed_historical['date_range']['start'] if processed_historical else 'N/A'} to {processed_historical['date_range']['end'] if processed_historical else 'N/A'}")
            
            else:
                print(f"  Failed to get data for {country}")
            
            time.sleep(1) 
        
        return all_data
    
    def save_to_file(self, data, filename):
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
        print(f"Data saved to {filename}")

def main():
    covid_api = COVID19API()

    countries = [
        "indonesia",
        "usa", 
        "china",
        "japan",
        "germany",
        "india",
        "brazil",
        "uk"
    ]
    
    print("="*70)
    print("COVID-19 API SCRAPER (disease.sh)")
    print("="*70)

    print("\nGetting global COVID-19 data...")
    global_data = covid_api.get_global_data()
    
    if global_data:
        print(f"Global Cases: {global_data.get('cases', 0):,}")
        print(f"Global Deaths: {global_data.get('deaths', 0):,}")
        print(f"Global Recovered: {global_data.get('recovered', 0):,}")
        print(f"Global Active: {global_data.get('active', 0):,}")
        print(f"Last Updated: {datetime.fromtimestamp(global_data.get('updated', 0) / 1000).strftime('%Y-%m-%d %H:%M:%S')}")

    print(f"\nScraping country-specific data...")
    country_data = covid_api.scrape_countries_data(countries)

    final_data = {
        'metadata': {
            'scraped_at': datetime.now().isoformat(),
            'countries_count': len(countries),
            'api_source': 'https://disease.sh/v3/covid-19',
            'description': 'COVID-19 data with historical timeline'
        },
        'global_data': global_data,
        'country_data': country_data
    }

    covid_api.save_to_file(final_data, 'covid19_api_data.json')

    print(f"\n{'='*70}")
    print("SCRAPING SUMMARY")
    print(f"{'='*70}")
    
    total_records = sum(
        data['historical_data']['total_records'] 
        for data in country_data.values() 
        if data.get('historical_data')
    )
    
    print(f"Total countries scraped: {len(country_data)}")
    print(f"Total historical records: {total_records:,}")
    print(f"Data saved to: covid19_api_data.json")

    if country_data:
        print(f"\nTop countries by confirmed cases:")
        sorted_countries = sorted(
            country_data.items(),
            key=lambda x: x[1]['current_stats'].get('cases', 0),
            reverse=True
        )
        
        for i, (country_key, data) in enumerate(sorted_countries[:5], 1):
            country_name = data['country_info']['country']
            cases = data['current_stats'].get('cases', 0)
            deaths = data['current_stats'].get('deaths', 0)
            cfr = data['statistics'].get('case_fatality_rate', 0)
            population = data['country_info'].get('population', 0)
            
            print(f"{i}. {country_name}")
            print(f"   Cases: {cases:,}")
            print(f"   Deaths: {deaths:,}")
            print(f"   Population: {population:,}")
            print(f"   Case Fatality Rate: {cfr}%")

    if country_data:
        first_country = list(country_data.keys())[0]
        sample_data = country_data[first_country]
        
        print(f"\nSample data structure for {sample_data['country_info']['country']}:")
        print(f"Current cases: {sample_data['current_stats'].get('cases', 0):,}")
        print(f"Today's new cases: {sample_data['current_stats'].get('today_cases', 0):,}")
        print(f"Case fatality rate: {sample_data['statistics'].get('case_fatality_rate', 0)}%")
        
        if sample_data.get('historical_data') and sample_data['historical_data']['daily_data']:
            print(f"Historical records: {sample_data['historical_data']['total_records']}")
            print(f"Date range: {sample_data['historical_data']['date_range']['start']} to {sample_data['historical_data']['date_range']['end']}")

            latest_historical = sample_data['historical_data']['daily_data'][-1]
            print(f"\nLatest historical data point:")
            print(f"Date: {latest_historical['date']}")
            print(f"Total cases: {latest_historical['total_cases']:,}")
            print(f"New cases: {latest_historical['new_cases']:,}")
            print(f"Active cases: {latest_historical['active']:,}")

if __name__ == "__main__":
    main()

COVID-19 API SCRAPER (disease.sh)

Getting global COVID-19 data...
Global Cases: 704,753,890
Global Deaths: 7,010,681
Global Recovered: 675,619,811
Global Active: 22,123,398
Last Updated: 2025-09-19 06:34:43

Scraping country-specific data...
Scraping COVID-19 data for countries: indonesia, usa, china, japan, germany, india, brazil, uk

Processing country 1/8: indonesia
  Total cases: 6,829,221
  Deaths: 162,063
  Historical records: 1143
  Date range: 1/1/21 to 9/9/22

Processing country 2/8: usa
  Total cases: 111,820,082
  Deaths: 1,219,487
  Historical records: 1143
  Date range: 1/1/21 to 9/9/22

Processing country 3/8: china
  Total cases: 503,302
  Deaths: 5,272
  Historical records: 1143
  Date range: 1/1/21 to 9/9/22

Processing country 4/8: japan
  Total cases: 33,803,572
  Deaths: 74,694
  Historical records: 1143
  Date range: 1/1/21 to 9/9/22

Processing country 5/8: germany
  Total cases: 38,828,995
  Deaths: 183,027
  Historical records: 1143
  Date range: 1/1/21 to 9/9/